# Punktwolkenvergleich – Genauigkeit und Vollständigkeit
Dieses Notebook lädt TLS- und UAV-Punktwolken, entfernt den Boden, berechnet Cloud-to-Cloud-Genauigkeit und voxelbasierte Vollständigkeit. Die Diagramme helfen bei der Bewertung der SfM-Rekonstruktion im Vergleich zur TLS-Referenz.

In [ ]:
import open3d as o3d
import pandas as pd
import matplotlib.pyplot as plt
from pointcloud_analysis import read_las_to_pcd, remove_ground, voxel_completeness, c2c_statistics

In [ ]:
# Parameter
VOXEL_SIZE = 0.001 
GROUND_HEIGHT_THRESHOLD = 0.1 

# Punktwolken laden
tls_path = r"C:\_Data\BTh11\BTh11_Trieblaengenwachstum\07_Check_PG-PointCloud_Completeness\70_BaseDate\PointCloud\20250319_RTC360_Kirsche_clipped.las"
uav_path = r"C:\_Data\BTh11\BTh11_Trieblaengenwachstum\07_Check_PG-PointCloud_Completeness\70_BaseDate\PointCloud\20250319_DJIMini3Pro_Kirsche_Metashape_clipped.las"

pcd_tls = read_las_to_pcd(tls_path)
pcd_uav = read_las_to_pcd(uav_path)

# Boden entfernen
pcd_tls = remove_ground(pcd_tls, GROUND_HEIGHT_THRESHOLD)
pcd_uav = remove_ground(pcd_uav, GROUND_HEIGHT_THRESHOLD)

In [ ]:
# Berechnungen
completeness = voxel_completeness(pcd_tls, pcd_uav, VOXEL_SIZE)
accuracy_stats = c2c_statistics(pcd_uav, pcd_tls)

# Tabelle erzeugen
df = pd.DataFrame({
    'Metrik': ['Mean [m]', 'Std [m]', 'RMSE [m]', 'Min [m]', 'Max [m]', 'N Punkte', 'Completeness [%]'],
    'Wert': [
        accuracy_stats['mean'],
        accuracy_stats['std'],
        accuracy_stats['rmse'],
        accuracy_stats['min'],
        accuracy_stats['max'],
        accuracy_stats['count'],
        completeness
    ]
})
df

In [ ]:
# Histogramm der C2C-Distanzen
plt.hist(accuracy_stats['distances'], bins=100, color='gray')
plt.title("C2C-Distanzen (UAV → TLS)")
plt.xlabel("Distanz [m]")
plt.ylabel("Anzahl Punkte")
plt.grid()
plt.tight_layout()
plt.show()